In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import datetime
from time import sleep

### Ouverture d'un navigateur chrome controlé par Python

In [2]:
chromedriver_path = "C:/Users/elisa/Projet Python/chromedriver_win32/chromedriver.exe"
#chromedriver_path = 'C:/Users/José/Documents/ENSAE Paris/2A/Python Pour le Data Scientist/chromedriver_win32/chromedriver.exe'

In [3]:
driver = webdriver.Chrome(chromedriver_path)

### Fonctions permettant de récupérer la compagnie aérienne, le nombre d'escales, le prix et la durée de vol

In [ ]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list

Pour un vol, la fonction get_airlines va récupérer "compagnie1" si la compagnie est identique à l'aller et au retour et "compagnie1, compagnie2" si elles sont différentes. La fonction remanier permet de récupérer pour un vol "compgnie1", "compagnie1" si on est dans le premier cas et "compagnie1", "compagnie2" dans le second cas

In [5]:
def remanier(compagnie):
    liste = []
    for i in range(len(compagnie)):
        if  ',' in compagnie[i] : 
            j= 0
            while compagnie[i][j]!=',' :
                j+=1
            liste.append(compagnie[i][:j])
            liste.append(compagnie[i][j+1:])
        else :
            liste.append(compagnie[i])
            liste.append(compagnie[i])
    return liste

La fonction remanier_2 permet de dédoubler le prix pour l'aller et le retour afin qu'il n'y ait pas de problème de longueur d'array et que le DataFrame puisse bien se former au fur et à mesure du scrapping.

In [6]:
def remanier_2(prix):
    liste = []
    for i in range(len(prix)):
        liste.append(prix[i])
        liste.append(prix[i])
    return liste

### Place au Scrapping

In [7]:
depart = ["PAR"]
Destinations = ["MAD", "SVO", "ATH", "OSL", "LON"]
date_aller = ["04/01/2022"]
date_retour = ["04/08/2022"]
date_a = pd.to_datetime(date_aller)
date_r = pd.to_datetime(date_retour)
result = pd.DataFrame([])

for k in range(1): 
    new_date_a = date_a + datetime.timedelta(days = 7*k)
    new_date_r = date_r + datetime.timedelta(days = 7*k)
    date_time_a = new_date_a.astype(str)
    date_time_r = new_date_r.astype(str)
    destination = ["SVO"]
    date_aller = [date_time_a]        
    date_retour = [date_time_r]
    url = f"https://www.kayak.fr/flights/{depart[0]}-{destination[0]}/{date_aller[0]}/{date_retour[0]}"
    driver.get(url)
    sleep(15)
    
    try:
        show_more_button = driver.find_element_by_xpath('//a[@class = "moreButton"]')
    except:
        input("Please solve the captcha then enter anything here to resume scraping.")

    while True:
        try:
            show_more_button.click()
            driver.implicitly_wait(10)
        except:
            break


    soup = BeautifulSoup(driver.page_source, 'html.parser')
    compagnie = remanier(get_airlines(soup))
    total_escales = get_total_stops(soup)
    prix = remanier_2(get_price(soup))
    duree = get_duration(soup)
    df = pd.DataFrame({'Compagnie':compagnie, "Total d'escales":total_escales , 'Prix': prix, 'Duree': duree}, 
                        columns = ['Compagnie', "Total d'escales", 'Prix', 'Duree'])
    df["Destination"] = "Moscou"
    df["Distance"] = 2486.34
    df["Date_aller"] = 20220401
    df["Date_retour"] = 20220408
    df["Date_aller"] = pd.to_datetime(df["Date_aller"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    df["Date_retour"] = pd.to_datetime(df["Date_retour"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    result = pd.concat([result, df], axis = 0)

Pour une semaine et pour Paris-Moscou on obtient le DataFrame suivant

In [9]:
#result.to_excel('moscou.xlsx', index = False) 
result

,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
0,Aeroflot,\ndirect,\n427 €,\n3h 50min,Moscou,2486.34,2022-04-01,2022-04-08
1,Air France,\ndirect,\n427 €,\n4h 05min,Moscou,2486.34,2022-04-01,2022-04-08
2,LOT,\n1 escale,\n228 €,\n5h 40min,Moscou,2486.34,2022-04-01,2022-04-08
3,Finnair,\n1 escale,\n228 €,\n5h 35min,Moscou,2486.34,2022-04-01,2022-04-08
4,Aeroflot,\ndirect,\n427 €,\n4h 00min,Moscou,2486.34,2022-04-01,2022-04-08
...,...,...,...,...,...,...,...,...
435,Aeroflot,\n1 escale,\n3 171 €,\n7h 15min,Moscou,2486.34,2022-04-01,2022-04-08
436,Brussels Airlines,\n1 escale,\n329 €,\n5h 55min,Moscou,2486.34,2022-04-01,2022-04-08
437,LOT,\n1 escale,\n329 €,\n14h 15min,Moscou,2486.34,2022-04-01,2022-04-08
438,Plusieurs compagnies,\n1 escale,\n1 584 €,\n5h 55min,Moscou,2486.34,2022-04-01,2022-04-08


### Combinaison de tous les fichiers excel récupérés

In [ ]:
fichiers = ["C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/oslo.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/oslo_2.xlsx",
            "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/moscou.xlsx","C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/madrid.xlsx",
            "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/madrid_2.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/londres_1.xlsx",
            "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/londres_2.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/athenes.xlsx"]
data_combined= pd.DataFrame()

for fichier in fichiers:
    df=pd.read_excel(fichier)
    data_combined= data_combined.append(df, ignore_index= True)

In [1]:
#data_combined

In [2]:
#data_combined.to_excel('data_combined.xlsx')